# Find the number of bus stop per gush

In [58]:
from pyproj import CRS
from pyproj import Transformer
import requests
import json
import pandas as pd
import csv
import time


def transform_itmcoordinate2itm(x,y):
    cc_from = CRS("IGD05/12")
    cc_to = CRS("EPSG:6991")
    trans = Transformer.from_crs(cc_from, cc_to, always_xy=True)
    a = trans.transform(x, y)
#     print(a[::-1])
    return a


def identify_by_xy(x, y):
    request_payload = {"x":x,"y":y,"mapTolerance":100,
                       "layers":[{"LayerType":0,"LayerName":"sub_gush_all","LayerFilter":""}]}

    headers = {'content-type': 'application/json'}
    url = r'https://ags.govmap.gov.il/Identify/IdentifyByXY'
    coordinate_request = requests.post(url, data=json.dumps(request_payload), headers=headers)
    xy_info = coordinate_request.json()
#     print(xy_info)
    gush_number = xy_info['data'][0]['Result'][0]['tabs'][0]['fields'][0]['FieldValue']
    print(gush_number)
    return gush_number


def get_gushnumber_from_list(df_coord, filename, column_label, start=0):
    output_path = r'C:\Users\majac\Desktop\JohnBriceProject\PublicTransport\{}'.format(filename)
    new_df = pd.DataFrame(columns=column_label)
    file = open(output_path, "a", encoding="utf-8")
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(column_label)
    
#     lat = list(df_coord['stop_lat'])
#     lon = list(df_coord['stop_lon'])

    org_stop_id = list(df_coord['original_stop_id'])
#     for n in range(start, len(lat)):
#         start = time.time()
#         print(n)
# #         xy = transform_itmcoordinate2itm(lon[n], lat[n])
        
# #         print(xy)
#         transform = time.time()
    xy_coordinates = list(df_coord['xy_coordinates'])
    for n in range(len(xy_coordinates)):
        xy = xy_coordinates[n]
#         print('it took {} to transform the coordinates'.format(transform-start))
        try:
            bus_gush = identify_by_xy(xy[0], xy[1])
            original_stop_id = org_stop_id[n]
            get_gush = time.time()
#             print("it took {} to get stop's informations".format(get_gush-transform))
            print('bus_gush: {}'.format(bus_gush))
            csv_row = ([original_stop_id, bus_gush, xy])
            writer.writerow(csv_row)
        except Exception as e:
            print(e)
            bus_gush = 'not found'
            # print('issue')
            original_stop_id = org_stop_id[n]
            csv_row = ([original_stop_id, bus_gush, xy])
            writer.writerow(csv_row)
        if n % 100 == 0:
            file.flush()

The file I will need to determine the number of bus stops in each gush is the following one:
It contains the lat/lon parameters that will allow me to define in which gush th bus stop belongs.

I use www.govmap.gov.il. In this website there is a option to identify a location by its XY coordinates: 
https://ags.govmap.gov.il/Identify/IdentifyByXY
I set the layer to be gush ID to retrieve it in the response.

However, since my coordinates are in lat/lon, I will first convert them to xy ITM. 

In [2]:
col_name = ['original_stop_id', 'bus_gush', 'coordinate XY']
file_PublicTransport = r'C:\Users\majac\Desktop\JohnBriceProject\PublicTransport\stops.txt'
bus_stop_df1 = pd.read_csv(file_PublicTransport)
# print(bus_stop_df1)
test_stops = bus_stop_df1.loc[:10]
test_stops


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
0,1,38831,בי''ס בר לב/בן יהודה,רחוב:בן יהודה 76 עיר: כפר סבא רציף: קומה:,32.183921,34.917806,0,NaN,6900
1,2,38832,הרצל/צומת בילו,רחוב:הרצל עיר: קרית עקרון רציף: קומה:,31.870034,34.819541,0,NaN,469
2,3,38833,הנחשול/הדייגים,רחוב:הנחשול 30 עיר: ראשון לציון רציף: קומה:,31.984553,34.782828,0,NaN,8300
3,4,38834,פריד/ששת הימים,רחוב:משה פריד 9 עיר: רחובות רציף: קומה:,31.888550,34.790904,0,NaN,8400
4,6,38836,ת. מרכזית לוד/הורדה,רחוב: עיר: לוד רציף: קומה:,31.956392,34.898098,0,NaN,7000
5,7,38837,חנה אברך/וולקני,רחוב:חנה אברך 9 עיר: רחובות רציף: קומה:,31.892166,34.796071,0,NaN,8400
6,8,38838,הרצל/משה שרת,רחוב:הרצל 20 עיר: קרית עקרון רציף: קומה:,31.857565,34.824106,0,NaN,469
7,9,38839,הבנים/אלי כהן,רחוב:הבנים 4 עיר: קרית עקרון רציף: קומה:,31.862305,34.821857,0,NaN,469
8,10,38840,ויצמן/הבנים,רחוב:וייצמן 11 עיר: קרית עקרון רציף: קומה:,31.865085,34.822237,0,NaN,469
9,11,38841,האירוס/הכלנית,רחוב:האירוס 13 עיר: קרית עקרון רציף: קומה:,31.865222,34.818957,0,NaN,469


In [15]:
get_gushnumber_from_list(bus_stop_df1, 'bus_stop_in_gush.csv', col_name)

0
6432
bus_gush: 6432
1
3797
bus_gush: 3797
2
6826
bus_gush: 6826
3
3656
bus_gush: 3656
4
5999
bus_gush: 5999
5
3655
bus_gush: 3655
6
4549
bus_gush: 4549
7
5105
bus_gush: 5105
8
3796
bus_gush: 3796
9
5101
bus_gush: 5101
10
5101
bus_gush: 5101
11
5105
bus_gush: 5105
12
3820
bus_gush: 3820
13
3820
bus_gush: 3820
14
5118
bus_gush: 5118
15
8001
bus_gush: 8001
16
8001
bus_gush: 8001
17
3688
bus_gush: 3688
18
4235
bus_gush: 4235
19
5107
bus_gush: 5107
20
3816
bus_gush: 3816
21
8001
bus_gush: 8001
22
8688
bus_gush: 8688
23
8688
bus_gush: 8688
24
8684
bus_gush: 8684
25
8684
bus_gush: 8684
26
8684
bus_gush: 8684
27
6703
bus_gush: 6703
28
6878
bus_gush: 6878
29
3698
bus_gush: 3698
30
8481
bus_gush: 8481
31
8310
bus_gush: 8310
32
3769
bus_gush: 3769
33
4536
bus_gush: 4536
34
3704
bus_gush: 3704
35
3694
bus_gush: 3694
36
6321
bus_gush: 6321
37
9090
bus_gush: 9090
38
5421
bus_gush: 5421
39
5421
bus_gush: 5421
40
4585
bus_gush: 4585
41
4586
bus_gush: 4586
42
4586
bus_gush: 4586
43
4586
bus_gush: 458

8233
bus_gush: 8233
266
8233
bus_gush: 8233
267
8262
bus_gush: 8262
268
8263
bus_gush: 8263
269
8264
bus_gush: 8264
270
8270
bus_gush: 8270
271
8264
bus_gush: 8264
272
8267
bus_gush: 8267
273
8269
bus_gush: 8269
274
8270
bus_gush: 8270
275
8267
bus_gush: 8267
276
8267
bus_gush: 8267
277
8267
bus_gush: 8267
278
8267
bus_gush: 8267
279
8273
bus_gush: 8273
280
8272
bus_gush: 8272
281
8273
bus_gush: 8273
282
8273
bus_gush: 8273
283
8272
bus_gush: 8272
284
8265
bus_gush: 8265
285
8266
bus_gush: 8266
286
8267
bus_gush: 8267
287
8269
bus_gush: 8269
288
8267
bus_gush: 8267
289
8266
bus_gush: 8266
290
8233
bus_gush: 8233
291
8266
bus_gush: 8266
292
8267
bus_gush: 8267
293
8267
bus_gush: 8267
294
8232
bus_gush: 8232
295
8267
bus_gush: 8267
296
8232
bus_gush: 8232
297
8267
bus_gush: 8267
298
8267
bus_gush: 8267
299
8267
bus_gush: 8267
300
8267
bus_gush: 8267
301
8274
bus_gush: 8274
302
8273
bus_gush: 8273
303
8273
bus_gush: 8273
304
8273
bus_gush: 8273
305
8274
bus_gush: 8274
306
8267
bus_gush: 8

8315
bus_gush: 8315
536
5025
bus_gush: 5025
537
6376
bus_gush: 6376
538
6376
bus_gush: 6376
539
9089
bus_gush: 9089
540
6256
bus_gush: 6256
541
6189
bus_gush: 6189
542
8564
bus_gush: 8564
543
8446
bus_gush: 8446
544
7923
bus_gush: 7923
545
8383
bus_gush: 8383
546
6191
bus_gush: 6191
547
7958
bus_gush: 7958
548
8273
bus_gush: 8273
549
5457
bus_gush: 5457
550
7923
bus_gush: 7923
551
8027
bus_gush: 8027
552
7923
bus_gush: 7923
553
7922
bus_gush: 7922
554
7922
bus_gush: 7922
555
7921
bus_gush: 7921
556
7922
bus_gush: 7922
557
7922
bus_gush: 7922
558
7922
bus_gush: 7922
559
7921
bus_gush: 7921
560
7921
bus_gush: 7921
561
8936
bus_gush: 8936
562
8936
bus_gush: 8936
563
8931
bus_gush: 8931
564
7706
bus_gush: 7706
565
8504
bus_gush: 8504
566
8502
bus_gush: 8502
567
9052
bus_gush: 9052
568
9051
bus_gush: 9051
569
8502
bus_gush: 8502
570
8504
bus_gush: 8504
571
9051
bus_gush: 9051
572
8462
bus_gush: 8462
573
8421
bus_gush: 8421
574
8808
bus_gush: 8808
575
7826
bus_gush: 7826
576
6368
bus_gush: 6

10012
bus_gush: 10012
841
10012
bus_gush: 10012
842
10009
bus_gush: 10009
843
10009
bus_gush: 10009
844
10009
bus_gush: 10009
845
10626
bus_gush: 10626
846
10620
bus_gush: 10620
847
10626
bus_gush: 10626
848
10620
bus_gush: 10620
849
10013
bus_gush: 10013
850
10626
bus_gush: 10626
851
10620
bus_gush: 10620
852
10620
bus_gush: 10620
853
10013
bus_gush: 10013
854
10049
bus_gush: 10049
855
10013
bus_gush: 10013
856
10001
bus_gush: 10001
857
10013
bus_gush: 10013
858
10611
bus_gush: 10611
859
10613
bus_gush: 10613
860
10001
bus_gush: 10001
861
10611
bus_gush: 10611
862
10014
bus_gush: 10014
863
10611
bus_gush: 10611
864
10014
bus_gush: 10014
865
10049
bus_gush: 10049
866
10048
bus_gush: 10048
867
10016
bus_gush: 10016
868
10036
bus_gush: 10036
869
10049
bus_gush: 10049
870
10049
bus_gush: 10049
871
10045
bus_gush: 10045
872
10049
bus_gush: 10049
873
10045
bus_gush: 10045
874
10671
bus_gush: 10671
875
10615
bus_gush: 10615
876
10036
bus_gush: 10036
877
10036
bus_gush: 10036
878
10615
bus_gu

10132
bus_gush: 10132
1136
10130
bus_gush: 10130
1137
8919
bus_gush: 8919
1138
12106
bus_gush: 12106
1139
12106
bus_gush: 12106
1140
8801
bus_gush: 8801
1141
12106
bus_gush: 12106
1142
12105
bus_gush: 12105
1143
10084
bus_gush: 10084
1144
12105
bus_gush: 12105
1145
12105
bus_gush: 12105
1146
8798
bus_gush: 8798
1147
10084
bus_gush: 10084
1148
8798
bus_gush: 8798
1149
HTTPSConnectionPool(host='ags.govmap.gov.il', port=443): Max retries exceeded with url: /Identify/IdentifyByXY (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000160FDC65E80>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))
1150
11984
bus_gush: 11984
1151
10085
bus_gush: 10085
1152
10086
bus_gush: 10086
1153
10089
bus_gush: 10089
1154
10084
bus_gush: 10084
1155
8785
bus_gush: 8785
1156
12238
bus_gush

10910
bus_gush: 10910
1411
10808
bus_gush: 10808
1412
10767
bus_gush: 10767
1413
10808
bus_gush: 10808
1414
10750
bus_gush: 10750
1415
10743
bus_gush: 10743
1416
10749
bus_gush: 10749
1417
10750
bus_gush: 10750
1418
10749
bus_gush: 10749
1419
10749
bus_gush: 10749
1420
10749
bus_gush: 10749
1421
10808
bus_gush: 10808
1422
10808
bus_gush: 10808
1423
12533
bus_gush: 12533
1424
10910
bus_gush: 10910
1425
12508
bus_gush: 12508
1426
10807
bus_gush: 10807
1427
10806
bus_gush: 10806
1428
12432
bus_gush: 12432
1429
10806
bus_gush: 10806
1430
12433
bus_gush: 12433
1431
12433
bus_gush: 12433
1432
12327
bus_gush: 12327
1433
12721
bus_gush: 12721
1434
10747
bus_gush: 10747
1435
12721
bus_gush: 12721
1436
12328
bus_gush: 12328
1437
10747
bus_gush: 10747
1438
10744
bus_gush: 10744
1439
10749
bus_gush: 10749
1440
10745
bus_gush: 10745
1441
10745
bus_gush: 10745
1442
10745
bus_gush: 10745
1443
10746
bus_gush: 10746
1444
10745
bus_gush: 10745
1445
10906
bus_gush: 10906
1446
10906
bus_gush: 10906
1447
1

11567
bus_gush: 11567
1696
11567
bus_gush: 11567
1697
11567
bus_gush: 11567
1698
11586
bus_gush: 11586
1699
11586
bus_gush: 11586
1700
11571
bus_gush: 11571
1701
11572
bus_gush: 11572
1702
11571
bus_gush: 11571
1703
11567
bus_gush: 11567
1704
11567
bus_gush: 11567
1705
11572
bus_gush: 11572
1706
11572
bus_gush: 11572
1707
10427
bus_gush: 10427
1708
11574
bus_gush: 11574
1709
11585
bus_gush: 11585
1710
10570
bus_gush: 10570
1711
10572
bus_gush: 10572
1712
11120
bus_gush: 11120
1713
11120
bus_gush: 11120
1714
11120
bus_gush: 11120
1715
11120
bus_gush: 11120
1716
11039
bus_gush: 11039
1717
11021
bus_gush: 11021
1718
11049
bus_gush: 11049
1719
11014
bus_gush: 11014
1720
11011
bus_gush: 11011
1721
11011
bus_gush: 11011
1722
11011
bus_gush: 11011
1723
11010
bus_gush: 11010
1724
11009
bus_gush: 11009
1725
11010
bus_gush: 11010
1726
11010
bus_gush: 11010
1727
11011
bus_gush: 11011
1728
10256
bus_gush: 10256
1729
10256
bus_gush: 10256
1730
10514
bus_gush: 10514
1731
10504
bus_gush: 10504
1732
1

12337
bus_gush: 12337
1999
11190
bus_gush: 11190
2000
12434
bus_gush: 12434
2001
11676
bus_gush: 11676
2002
10870
bus_gush: 10870
2003
10200
bus_gush: 10200
2004
10764
bus_gush: 10764
2005
10910
bus_gush: 10910
2006
list index out of range
2007
11547
bus_gush: 11547
2008
10013
bus_gush: 10013
2009
10428
bus_gush: 10428
2010
11562
bus_gush: 11562
2011
11562
bus_gush: 11562
2012
11561
bus_gush: 11561
2013
11561
bus_gush: 11561
2014
11534
bus_gush: 11534
2015
10445
bus_gush: 10445
2016
10444
bus_gush: 10444
2017
11578
bus_gush: 11578
2018
10923
bus_gush: 10923
2019
10096
bus_gush: 10096
2020
11394
bus_gush: 11394
2021
11394
bus_gush: 11394
2022
10423
bus_gush: 10423
2023
11237
bus_gush: 11237
2024
17793
bus_gush: 17793
2025
11508
bus_gush: 11508
2026
11508
bus_gush: 11508
2027
17145
bus_gush: 17145
2028
10906
bus_gush: 10906
2029
10804
bus_gush: 10804
2030
10804
bus_gush: 10804
2031
8831
bus_gush: 8831
2032
8845
bus_gush: 8845
2033
8845
bus_gush: 8845
2034
8840
bus_gush: 8840
2035
11136
b

KeyboardInterrupt: 

In [19]:
get_gushnumber_from_list(bus_stop_df1, 'bus_stop_in_gush.csv', col_name, start=28000)

28000
10571
bus_gush: 10571
28001
10571
bus_gush: 10571
28002
10571
bus_gush: 10571
28003
10571
bus_gush: 10571
28004
list index out of range
28005
list index out of range
28006
38352
bus_gush: 38352
28007
100622
bus_gush: 100622
28008
38491
bus_gush: 38491
28009
38491
bus_gush: 38491
28010
38491
bus_gush: 38491
28011
38490
bus_gush: 38490
28012
38490
bus_gush: 38490
28013
38490
bus_gush: 38490
28014
38490
bus_gush: 38490
28015
38493
bus_gush: 38493
28016
38493
bus_gush: 38493
28017
38493
bus_gush: 38493
28018
38491
bus_gush: 38491
28019
6490
bus_gush: 6490
28020
6891
bus_gush: 6891
28021
38212
bus_gush: 38212
28022
17610
bus_gush: 17610
28023
17507
bus_gush: 17507
28024
17748
bus_gush: 17748
28025
16651
bus_gush: 16651
28026
17833
bus_gush: 17833
28027
17835
bus_gush: 17835
28028
1222
bus_gush: 1222
28029
100145
bus_gush: 100145
28030
100145
bus_gush: 100145
28031
100145
bus_gush: 100145
28032
39073
bus_gush: 39073
28033
39911
bus_gush: 39911
28034
100422
bus_gush: 100422
28035
100422

In [61]:
file_stop_gush = r'C:\Users\majac\Desktop\JohnBriceProject\PublicTransport\bus_stop_in_gushDF.csv'
gush_busStopDF = pd.read_csv(file_stop_gush)
gush_busStopDF.columns = ['original_stop_id', 'bus_gush', 'xy_coordinates', '']
gush_busStopDF.iloc[9940:9945]



,original_stop_id,bus_gush,xy_coordinates,
9940,13527,7060,"(179303.90007467792, 662623.1613116766)",NaN
9941,13528,not found,"(179145.3088320659, 662783.3610445282)",NaN
9942,13529,not found,"(179207.79067064394, 662774.2428068236)",NaN
9943,13531,8940,"(179178.47194790325, 663074.1004140964)",NaN
9944,13532,8940,"(179219.631675508, 663022.7056340366)",NaN


About 10% of the results came back empty. I will re-run them to check whether the problem comes from overloading server.

In [49]:
gush_busStopDF.set_index('bus_gush', inplace=True)
bus_stopU = gush_busStopDF.loc['not found']
bus_stopU



,original_stop_id,xy_coordinates,
bus_gush,,,
not found,210,"(185837.43376203475, 687735.8165967626)",NaN
not found,234,"(186271.54161313217, 687280.2661323904)",NaN
not found,251,"(186651.92551056843, 688053.9599645432)",NaN
not found,310,"(186725.42687535673, 691345.5430790897)",NaN
not found,331,"(187084.94318129966, 692510.7856540857)",NaN
...,...,...,...
not found,44074,"(227744.40741678292, 636861.0062027134)",NaN
not found,44103,"(224895.76033022866, 641335.7731961773)",NaN
not found,44104,"(224873.70987266224, 641337.7574758393)",NaN


In [59]:
bus_stopU.reset_index
get_gushnumber_from_list(bus_stopU, 'bus_stop_NotFound.csv', col_name)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [67]:
gush_busStopDF.drop_duplicates(subset='original_stop_id')

    # gush_busStopDF.set_index('bus_gush', inplace=True)
bus_gushGB = gush_busStopDF.groupby('bus_gush').count()
bus_gushGB.drop(columns=['xy_coordinates', ""], inplace=True)
file_stop_gush = r'C:\Users\majac\Desktop\JohnBriceProject\PublicTransport\GushAndBus_stopFinal.csv'
bus_gushGB.to_csv(file_stop_gush)

bus_gushGB


,original_stop_id
bus_gush,
100001,3
100005,1
10001,3
100011,5
100012,6
...,...
9908,1
9909,5
995,2
